In [66]:
# simple NN test
import numpy as np

# input a_k -- Wkj -- g(in_j) -  hidden a_j -- Wji -- g(in_i) - output a_i - y_i
# hidden to output layer
# Wji = Wji + alpha (yi-ai)g'(ini)aj ini=sum over j Wji aj
# input to hidden layer
# Wkj = Wkj + alpha sum over i (yi-ai) Wji g'(inj) ak inj = sum over k Wkj ak

def sigmoid(x,deriv=False):
    if(deriv==True):
        return x*(1-x)
    np.putmask(x, x > 100, 100) # limit array values of x (np.putmask(a, a >= m, m - 1))
    np.putmask(x, x < -100, -100)
    return 1/(1+np.exp(-x))

alpha = -0.7
# solid = 1111, vertical = 1010 or 0101, diagonal = 1001 or 0110, horizontal = 1100 or 0011
# bias as last element
solid = np.array([[1,1,1,1,1]]) # solid = 1111 : last element for bias always 1
vert0 = np.array([[1,0,1,0,1]]) # vertical = 1010
vert1 = np.array([[0,1,0,1,1]]) # vertical = 0101
diag0 = np.array([[1,0,0,1,1]]) # diagonal = 1001
diag1 = np.array([[0,1,1,0,1]]) # diagonal = 0110
hor0 = np.array([[1,1,0,0,1]]) # horizontal = 1100
hor1 = np.array([[0,0,1,1,1]]) # horizontal = 0011

# target outputs
tsol = np.array([[1,0,0,0]]) # solid
tvert = np.array([[0,1,0,0]]) # vertical
tdiag = np.array([[0,0,1,0]]) # diagonal
thor = np.array([[0,0,0,1]]) # horizontal

wji = np.random.rand(4,4) * 0.1 - 0.05
wkj = np.random.rand(5,4) * 0.1 - 0.05 # bias trick: last line

np.set_printoptions(precision=3)

for i in xrange(10):
    ak = vert0
    yi = tvert
    wkj[4] = np.random.rand(4) * 0.1 - 0.05 # set bias
    inj = np.dot(ak,wkj)
    aj = sigmoid(inj,False)
    ini = np.dot(aj,wji)
    ai = sigmoid(ini,False)
    deltai = (yi-ai)
    wji += alpha * aj.T * deltai * sigmoid(ini, True) # Wji += alpha (yi-ai)g'(ini)aj 
    wkj += alpha * np.dot(ak.T,np.dot(sigmoid(inj, True), np.dot(wji, deltai.T))) # Wkj += alpha ak g'(inj) sum i Wji(yi-ai)

# see if its working: predicted class correct?
ak = vert0
inj = np.dot(ak,wkj)
aj = sigmoid(inj,False)
ini = np.dot(aj,wji)
ai = sigmoid(ini,False)
print ai

[[0.621 0.5   0.398 0.548]]


In [90]:
# residual NN test - default no fb as benchmark
import numpy as np

# input - wi0 - w01 - w12 - w2o - output
# residual         w02

def sigmoid(x,deriv=False):
    if(deriv==True):
        return x*(1-x)
    np.putmask(x, x > 100, 100) # limit array values of x (np.putmask(a, a >= m, m - 1))
    np.putmask(x, x < -100, -100)
    return 1/(1+np.exp(-x))

etha = 0.1
# solid = 1111, vertical = 1010 or 0101, diagonal = 1001 or 0110, horizontal = 1100 or 0011
# bias as last element
solid = np.array([[1,1,1,1,1]]) # solid = 1111 : last element for bias always 1
vert0 = np.array([[1,0,1,0,1]]) # vertical = 1010
vert1 = np.array([[0,1,0,1,1]]) # vertical = 0101
diag0 = np.array([[1,0,0,1,1]]) # diagonal = 1001
diag1 = np.array([[0,1,1,0,1]]) # diagonal = 0110
hor0 = np.array([[1,1,0,0,1]]) # horizontal = 1100
hor1 = np.array([[0,0,1,1,1]]) # horizontal = 0011

# target outputs
tsol = np.array([[1,0,0,0]]) # solid
tvert = np.array([[0,1,0,0]]) # vertical
tdiag = np.array([[0,0,1,0]]) # diagonal
thor = np.array([[0,0,0,1]]) # horizontal

wi0 = np.random.rand(5,5) * 0.1 - 0.05
w01 = np.random.rand(5,5) * 0.1 - 0.05
w12 = np.random.rand(5,5) * 0.1 - 0.05
w2o = np.random.rand(5,4) * 0.1 - 0.05
w02 = np.random.rand(5,5) * 0.1 - 0.05

np.set_printoptions(precision=3)

for i in xrange(100):
    # forward pass
    a0 = diag0
    y0 = tdiag
    wi0[4] = np.random.rand(5) * 0.1 - 0.05 # set bias
    w01[4] = np.random.rand(5) * 0.1 - 0.05 # set bias
    w12[4] = np.random.rand(5) * 0.1 - 0.05 # set bias
    a1 = sigmoid(np.dot(a0, wi0), False)
    a2 = sigmoid(np.dot(a1, w01), False)
    a3 = sigmoid(np.dot(a2, w12), False)
    a4 = sigmoid(np.dot(a3, w2o), False)
    deltao = (a4 - y0)
    # backprop and weight updates
    w2o += etha * np.clip(a3.T * deltao * sigmoid(np.dot(a3, w2o), True), -5., 5.)
    w12 += etha * np.clip(a2.T * np.dot(deltao * sigmoid(np.dot(a3, w2o), True), w2o.T) * sigmoid(np.dot(a2, w12), True), -5., 5.)
    w01 += etha * np.clip(a1.T * np.dot(deltao * sigmoid(np.dot(a3, w2o), True), w2o.T) * sigmoid(np.dot(a2, w12), True) * w12 * sigmoid(np.dot(a1, w01), True), -5., 5.)
    wi0 += etha * np.clip(a0.T * np.dot(deltao * sigmoid(np.dot(a3, w2o), True), w2o.T) * sigmoid(np.dot(a2, w12), True) * w12 * sigmoid(np.dot(a1, w01), True) * w01 * sigmoid(np.dot(a0, wi0), True), -5., 5.)

# see if its working: predicted class correct?
a0 = diag0
a1 = sigmoid(np.dot(a0, wi0), False)
a2 = sigmoid(np.dot(a1, w01), False)
a3 = sigmoid(np.dot(a2, w12), False)
a4 = sigmoid(np.dot(a3, w2o), False)
print a4

[[0.734 0.009 0.501 0.106]]


In [137]:
# residual NN test
import numpy as np

# input - wi0 - w01 - w12 - w2o - output
# residual         w02

def sigmoid(x,deriv=False):
    if(deriv==True):
        return x*(1-x)
    np.putmask(x, x > 100, 100) # limit array values of x (np.putmask(a, a >= m, m - 1))
    np.putmask(x, x < -100, -100)
    return 1/(1+np.exp(-x))

etha = -0.2
# solid = 1111, vertical = 1010 or 0101, diagonal = 1001 or 0110, horizontal = 1100 or 0011
# bias as last element
solid = np.array([[1,1,1,1,1]]) # solid = 1111 : last element for bias always 1
vert0 = np.array([[1,0,1,0,1]]) # vertical = 1010
vert1 = np.array([[0,1,0,1,1]]) # vertical = 0101
diag0 = np.array([[1,0,0,1,1]]) # diagonal = 1001
diag1 = np.array([[0,1,1,0,1]]) # diagonal = 0110
hor0 = np.array([[1,1,0,0,1]]) # horizontal = 1100
hor1 = np.array([[0,0,1,1,1]]) # horizontal = 0011

# target outputs
tsol = np.array([[1,0,0,0]]) # solid
tvert = np.array([[0,1,0,0]]) # vertical
tdiag = np.array([[0,0,1,0]]) # diagonal
thor = np.array([[0,0,0,1]]) # horizontal

wi0 = np.random.rand(5,5) * 0.1 - 0.05
w01 = np.random.rand(5,5) * 0.1 - 0.05
w12 = np.random.rand(5,5) * 0.1 - 0.05
w2o = np.random.rand(5,4) * 0.1 - 0.05
w02 = np.random.rand(5,5) * 0.1 - 0.05

np.set_printoptions(precision=3)

for i in xrange(100):
    # forward pass
    a0 = diag0
    y0 = tdiag
    wi0[4] = np.random.rand(5) * 0.1 - 0.05 # set bias
    w01[4] = np.random.rand(5) * 0.1 - 0.05 # set bias
    w12[4] = np.random.rand(5) * 0.1 - 0.05 # set bias
    a1 = sigmoid(np.dot(a0, wi0), False)
    a2 = sigmoid(np.dot(a1, w01), False)
    a3 = sigmoid(np.dot(a2, w12) + np.dot(a1, w02), False)
    a4 = sigmoid(np.dot(a3, w2o), False)
    deltao = (a4 - y0)
    # backprop and weight updates
    w2o += etha * np.clip(a3.T * deltao * sigmoid(np.dot(a3, w2o), True), -5., 5.)
    w12 += etha * np.clip(a2.T * np.dot(deltao * sigmoid(np.dot(a3, w2o), True), w2o.T) * sigmoid(np.dot(a2, w12) + np.dot(a1, w02), True), -5., 5.)
    w01 += etha * np.clip(a1.T * np.dot(deltao * sigmoid(np.dot(a3, w2o), True), w2o.T) * sigmoid(np.dot(a2, w12) + np.dot(a1, w02), True) * w12 * sigmoid(np.dot(a1, w01), True), -5., 5.)
    wi0 += etha * np.clip(a0.T * np.dot(deltao * sigmoid(np.dot(a3, w2o), True), w2o.T) * sigmoid(np.dot(a2, w12) + np.dot(a1, w02), True) * w12 * sigmoid(np.dot(a1, w01), True) * w01 * sigmoid(np.dot(a0, wi0), True), -5., 5.)
    w12 += etha * np.clip(a1.T * np.dot(deltao * sigmoid(np.dot(a3, w2o), True), w2o.T) * sigmoid(np.dot(a2, w12) + np.dot(a1, w02), True), -5., 5.)
    
# see if its working: predicted class correct?
a0 = diag0
a1 = sigmoid(np.dot(a0, wi0), False)
a2 = sigmoid(np.dot(a1, w01), False)
a3 = sigmoid(np.dot(a2, w12) + np.dot(a1, w02), False)
a4 = sigmoid(np.dot(a3, w2o), False)
print a4

[[0.5  0.5  0.73 0.5 ]]


In [146]:
# residual NN test
import numpy as np

# input - wi0 - w01 - w12 - w2o - output
# residual         w02

def sigmoid(x,deriv=False):
    if(deriv==True):
        return x*(1-x)
    np.putmask(x, x > 100, 100) # limit array values of x (np.putmask(a, a >= m, m - 1))
    np.putmask(x, x < -100, -100)
    return 1/(1+np.exp(-x))

etha = -0.2
# solid = 1111, vertical = 1010 or 0101, diagonal = 1001 or 0110, horizontal = 1100 or 0011
# bias as last element
solid = np.array([[1,1,1,1,1]]) # solid = 1111 : last element for bias always 1
vert0 = np.array([[1,0,1,0,1]]) # vertical = 1010
vert1 = np.array([[0,1,0,1,1]]) # vertical = 0101
diag0 = np.array([[1,0,0,1,1]]) # diagonal = 1001
diag1 = np.array([[0,1,1,0,1]]) # diagonal = 0110
hor0 = np.array([[1,1,0,0,1]]) # horizontal = 1100
hor1 = np.array([[0,0,1,1,1]]) # horizontal = 0011

# target outputs
tsol = np.array([[1,0,0,0]]) # solid
tvert = np.array([[0,1,0,0]]) # vertical
tdiag = np.array([[0,0,1,0]]) # diagonal
thor = np.array([[0,0,0,1]]) # horizontal

wi0 = np.random.rand(5,5) * 0.1 - 0.05
w01 = np.random.rand(5,5) * 0.1 - 0.05
w12 = np.random.rand(5,5) * 0.1 - 0.05
w2o = np.random.rand(5,4) * 0.1 - 0.05
w02 = np.random.rand(5,5) * 0.1 - 0.05

np.set_printoptions(precision=3)

for i in xrange(100):
    # forward pass
    a0 = vert0
    y0 = tvert
    wi0[4] = np.random.rand(5) * 0.1 - 0.05 # set bias
    w01[4] = np.random.rand(5) * 0.1 - 0.05 # set bias
    w12[4] = np.random.rand(5) * 0.1 - 0.05 # set bias
    a1 = sigmoid(np.dot(a0, wi0), False)
    a2 = sigmoid(np.dot(a1, w01), False)
    a3 = sigmoid(np.dot(a2, w12) + np.dot(a1, w02), False)
    a4 = sigmoid(np.dot(a3, w2o), False)
    deltao = (a4 - y0)
    # backprop and weight updates
    dwo = deltao * sigmoid(np.dot(a3, w2o), True)
    dw2 = np.dot(dwo, w2o.T) * sigmoid(np.dot(a2, w12) + np.dot(a1, w02), True)
    dw1 = dw2 * w12 * sigmoid(np.dot(a1, w01), True)
    dw0 = dw1 * w01 * sigmoid(np.dot(a0, wi0), True)
    w2o += etha * np.clip(a3.T * dwo, -5., 5.)
    w12 += etha * np.clip(a2.T * dw2, -5., 5.)
    w01 += etha * np.clip(a1.T * dw1, -5., 5.)
    wi0 += etha * np.clip(a0.T * dw0, -5., 5.)
    w12 += etha * np.clip(a1.T * dw2, -5., 5.)
       
# see if its working: predicted class correct?
a0 = vert0
a1 = sigmoid(np.dot(a0, wi0), False)
a2 = sigmoid(np.dot(a1, w01), False)
a3 = sigmoid(np.dot(a2, w12) + np.dot(a1, w02), False)
a4 = sigmoid(np.dot(a3, w2o), False)
print a4

[[0.5  0.73 0.5  0.5 ]]
